# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte SVM

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co
Modificado por Heisman Arcila. (Versión original en: https://github.com/jdariasl/ML_IntroductoryCourse/tree/master/Labs)

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Danny Francisco Hernandez Godoy
#### C.C: 1105784633
#### Segundo Integrante: Daniel Martinez Galeano
#### C.C: 1036645337


In [0]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("ignore")

## Ejercicio 1

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [0]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html. 

Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. 

Complete el siguiente código usando un clasificador basado en SVM:

In [0]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

#Validamos el modelo
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
porcentaje=np.zeros(Folds)
def invento(kernelio, ceste, gammeste):
  j = 0
  for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]

    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)

    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    modelo = SVC(kernel=kernelio, C=ceste, gamma=gammeste)
    modelo.fit(Xtrain, Ytrain)
    porcentaje[j]=(modelo.support_vectors_.shape[0])/Xtrain.shape[0] #cuantos vectores de soporte tiene encima de las muestras de entrenaimento

    #Validación
    Ytrain_pred = modelo.predict(Xtrain)
    Yest = modelo.predict(Xtest)

    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1

  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print(np.mean(porcentaje))
  return np.mean(EficienciaVal), np.std(EficienciaVal), np.mean(porcentaje)

In [4]:
EX=invento('linear', 1, 'auto')

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
0.26203839770590365


Realice los experimientos necesarios para llenar la siguiente tabla:

In [5]:
import pandas as pd
#import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
#Escriba acá su código

for i in df_types.index:
    print(i[0])
    if float(i[2])==0:
      ED=invento(i[0], i[1], 'auto')
      df_types["Eficiencia en validacion"][i] = float(ED[0])
      df_types["Intervalo de confianza"][i] = float(ED[1])
      df_types["% de Vectores de Soporte"][i] = float(ED[2])
    else:
      ED=invento(i[0], i[1], i[2])
      df_types["Eficiencia en validacion"][i] = float(ED[0])
      df_types["Intervalo de confianza"][i] = float(ED[1])
      df_types["% de Vectores de Soporte"][i] = float(ED[2])
#df_types["Eficiencia en validacion"][3] = "0.97077"
#df_types["Intervalo de confianza"][3] = "0.01548"
#df_types["% de Vectores de Soporte"][3] = "0.2620"
#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

linear
Eficiencia durante el entrenamiento = 0.9226595902460473+-0.01797959849011947
Eficiencia durante la validación = 0.7960012912715104+-0.053296257064013926
1.0
linear
Eficiencia durante el entrenamiento = 0.9967600864350297+-0.0007994779941787222
Eficiencia durante la validación = 0.9706994383727887+-0.029264745294069673
0.608322259431444
linear
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9804458869759817+-0.01661113527781064
0.28659534274928156
linear
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
0.26203839770590365
linear
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
0.26203839770590365
linear
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
0.26203839770590365
rbf
Eficiencia durante el entrenamiento = 0.31608949943

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.
#NO QGRID

In [6]:
df_types

Eficiencia en validacion  ... % de Vectores de Soporte
Kernel C       gamma                           ...                         
linear 0.001   0.00                  0.796001  ...                        1
       0.010   0.00                  0.970699  ...                 0.608322
       0.100   0.00                  0.980446  ...                 0.286595
       1.000   0.00                  0.970777  ...                 0.262038
       10.000  0.00                  0.970777  ...                 0.262038
       100.000 0.00                  0.970777  ...                 0.262038
rbf    0.001   0.01                  0.315623  ...                        1
               0.10                  0.277914  ...                        1
               1.00                  0.254171  ...                 0.999539
       0.010   0.01                  0.315623  ...                        1
               0.10                  0.277914  ...                        1
               1.00                  0.254171  ...                        1
       0.100   0.01                  0.832031  ...                        1
               0.10                  0.277914  ...                        1
               1.00                  0.254171  ...                        1
       1.000   0.01                  0.984659  ...                 0.643049
               0.10                  0.808549  ...                 0.972674
               1.00                  0.256964  ...                        1
       10.000  0.01                  0.990246  ...                 0.577322
               0.10                  0.815471  ...                 0.972213
               1.00                  0.258369  ...                        1
       100.000 0.01                  0.990246  ...                 0.577322
               0.10                  0.815471  ...                 0.972213
               1.00                  0.258369  ...                        1

[24 rows x 3 columns]

In [0]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                                                    
       0.010   0.00                                                    
       0.100   0.00                                                    
       1.000   0.00                   0.97077                0.01548   
       10.000  0.00                                                    
       100.000 0.00                                                    
rbf    0.001   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.010   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.100   0.01                                                    
               0.10                                                    
               1.00                                                    
       1.000   0.01                                                    
               0.10                                                    
               1.00                                                    
       10.000  0.01                                                    
               0.10                                                    
               1.00                                                    
       100.000 0.01                                                    
               0.10                                                    
               1.00                                                    

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                    0.2620  
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00

**Nota:**  *En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.*

## Ejercicio 2

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia **Uno vs el resto**, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

In [0]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
def TrainSVM_OnevsRest(celisio, kernelisio , gammita, X, Y):
    nClases = np.unique(Y).shape[0]
    modelos = []
    print(nClases)
    for i in range(nClases):
        yAux = np.copy(Y)
        yAux[Y==i] = 1
        yAux[Y!=i] = 0
        if(gammita != 0):
            modelo =  SVC(C=celisio, kernel=kernelisio, gamma=gammita, probability=True)
        else:
            modelo =  SVC(C=celisio, kernel=kernelisio, probability=True)
            
        modelo.fit(X, yAux) 
        modelos.append(modelo)
        
    return modelos


    

def ValidaSVM_OnevsRest(data, models, Y):
  Ytrain_pred = model.predict(data)
  #print((Ytrain_pred))
  #print(Y)
  #print(model)
  nModelos=np.unique(Y).shape[0]
  for i in range(nModelos):
    Yaux = np.copy(Y)
    Yaux[Y == i] = 1
    Yaux[Y != i] = 0
    #print(Yaux)
    a=Ytrain_pred
    b=Yaux
    if (a==b).all():
      return Ytrain_pred
      
def ValidaSVM_OnevsRest2(data, models):
  este=[]
  prediccion=[]
  print(models)
  for i in models:
    prediccion.append(i.predict(data))
  for k in range(len(data)):
    otro =[]
    #print(prediccion[0][0])
    for j in range(len(prediccion)):
      otro.append(prediccion[j][k])
    este.append(np.argmax(otro))
  return np.asarray(este)


  
def TrainSVM_OnevsRest2(c, Kernel, Gamma, X, Y):
    n_classes = np.unique(Y).shape[0]
    model = []
    print(n_classes)
    for i in range(n_classes):
        Yaux = np.copy(Y)
        Yaux[Y == i] = 1
        Yaux[Y != i] = 0
        clf = OneVsRestClassifier(SVC(C=c, kernel=Kernel , gamma=Gamma, probability=True))
        clf.fit(X, Yaux)
        model.append(clf)
    return model
    

## Ejercicio 3

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [69]:
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
def invento2(kernel, C, gamma):
  j = 0
  for train, test in skf.split(X, Y):
      Xtrain = X[train,:]
      Ytrain = Y[train]
      Xtest = X[test,:]
      Ytest = Y[test]

      #Normalizamos los datos
      scaler = preprocessing.StandardScaler().fit(Xtrain)
      Xtrain = scaler.transform(Xtrain)
      Xtest = scaler.transform(Xtest)

      #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento

      #C=1
      #gamma='auto'
      #kernel='linear'

      modelos = TrainSVM_OnevsRest(C, kernel, gamma, Xtrain, Ytrain)
      
      #print(modelo)

      #Validación
      modelo = modelos[j]
      porcentaje[j]=(modelo.support_vectors_.shape[0])/Xtrain.shape[0] #cuantos vectores de soporte tiene encima de las muestras de entrenaimento
      #porcentaje[j]=(modelo.support_vectors_.shape[0])/Xtrain.shape[0]
      #print(modelo)
      #print(Xtrain)
      #print((Ytrain))
      #Ytrain_pred = ValidaSVM_OnevsRest(Xtest, modelo, Ytest)
      #Yest = ValidaSVM_OnevsRest(Xtest, modelo, Ytest)
      
      Ytrain_pred = ValidaSVM_OnevsRest2(Xtrain, modelos)
      Yest = ValidaSVM_OnevsRest2(Xtest, modelos)
      
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain.ravel())
      EficienciaVal[j] = np.mean(Yest == Ytest.ravel())
      j += 1

  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print(np.mean(porcentaje))
  return np.mean(EficienciaVal), np.std(EficienciaVal), np.mean(porcentaje)
  #print(np.mean(porcentaje))
EXS=invento2('linear', 1,'auto')

4
[SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False), SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False), SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False), SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)]
[SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_

Realice los experimientos necesarios para llenar la siguiente tabla:

In [70]:
import pandas as pd
#import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
#Escriba acá su código
for i in df_types.index:
    print(i[0])
    if float(i[2])==0:
      ED=invento2(i[0], i[1], 'auto')
      df_types["Eficiencia en validacion"][i] = float(ED[0])
      df_types["Intervalo de confianza"][i] = float(ED[1])
      df_types["% de Vectores de Soporte"][i] = float(ED[2])
    else:
      ED=invento2(i[0], i[1], i[2])
      df_types["Eficiencia en validacion"][i] = float(ED[0])
      df_types["Intervalo de confianza"][i] = float(ED[1])
      df_types["% de Vectores de Soporte"][i] = float(ED[2])

# df_types["Eficiencia en validacion"][3] = "0.97633"
# df_types["Intervalo de confianza"][3] = "0.01837"
# df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

linear
4
[SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False), SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False), SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False), SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)]
[SVC(C=0.001, cache_size=200, class_weight=Non

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.
#NO QGRID

In [71]:
df_types

Eficiencia en validacion  ... % de Vectores de Soporte
Kernel C       gamma                           ...                         
linear 0.001   0.00                  0.247218  ...                 0.521287
       0.010   0.00                  0.912323  ...                 0.300888
       0.100   0.00                  0.941576  ...                 0.103692
       1.000   0.00                  0.937463  ...                0.0708331
       10.000  0.00                  0.937463  ...                0.0708331
       100.000 0.00                  0.937463  ...                0.0708331
rbf    0.001   0.01                  0.247218  ...                  0.50138
               0.10                  0.247218  ...                 0.508315
               1.00                  0.247218  ...                 0.660257
       0.010   0.01                  0.247218  ...                 0.545373
               0.10                  0.247218  ...                 0.973178
               1.00                  0.247218  ...                        1
       0.100   0.01                  0.247218  ...                 0.551854
               0.10                  0.247218  ...                 0.980115
               1.00                  0.247218  ...                        1
       1.000   0.01                  0.949787  ...                 0.343472
               0.10                  0.407261  ...                 0.968981
               1.00                  0.247218  ...                        1
       10.000  0.01                   0.96235  ...                 0.281914
               0.10                  0.447643  ...                 0.966672
               1.00                  0.247218  ...                        1
       100.000 0.01                   0.96235  ...                 0.281914
               0.10                  0.447643  ...                 0.966672
               1.00                  0.247218  ...                        1

[24 rows x 3 columns]

In [0]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                                                    
       0.010   0.00                                                    
       0.100   0.00                                                    
       1.000   0.00                   0.97633                0.01837   
       10.000  0.00                                                    
       100.000 0.00                                                    
rbf    0.001   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.010   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.100   0.01                                                    
               0.10                                                    
               1.00                                                    
       1.000   0.01                                                    
               0.10                                                    
               1.00                                                    
       10.000  0.01                                                    
               0.10                                                    
               1.00                                                    
       100.000 0.01                                                    
               0.10                                                    
               1.00                                                    

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                    0.2778  
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00

**Nota:**  *En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.*